In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, SpatialDropout1D, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
from gensim.models import KeyedVectors
import requests

In [35]:
!pip install gradio==4.32.0
!pip install tensorflow
# !pip install --upgrade tensorflow


In [3]:
# Tải các tài nguyên cần thiết từ NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Tải nội dung của tệp stop_words.py từ GitHub
url = 'https://raw.githubusercontent.com/trungtv/vivi_spacy/master/vi/vi/stop_words.py'
response = requests.get(url)
stop_words_content = response.text

In [54]:
# Phân tích nội dung của tệp stop_words.py để lấy danh sách stopwords
start_index = stop_words_content.find('"""') + 3
end_index = stop_words_content.rfind('"""')
stop_words_list = stop_words_content[start_index:end_index].split("\n")
# Tạo danh sách stopwords từ danh sách bạn vừa phân tích
stop_words = set(word.strip() for word in stop_words_list if word.strip())

In [55]:
# Tiền xử lý văn bản
def preprocess_text(text):
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word not in stop_words and word.isalnum()]
    return ' '.join(filtered_words)

In [56]:
# Load dữ liệu
df = pd.read_excel('/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/Finaldataset.xlsx')
df.dropna(axis=1, how='all', inplace=True)
contents = df['content'].tolist()
topics = df['label'].tolist()
contents = [str(content).strip() for content in contents]
processed_contents = [preprocess_text(content) for content in contents]

In [57]:
# Tiền xử lý nhãn
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(topics)
num_classes = len(set(y))
y = to_categorical(y, num_classes=num_classes)

In [58]:
# Tokenization và padding
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(processed_contents)
X = tokenizer.texts_to_sequences(processed_contents)
# Tìm độ dài của phần tử dài nhất trong danh sách X
max_length = max(len(seq) for seq in X)
# Padding các chuỗi số nguyên để có độ dài bằng nhau
X = pad_sequences(X, maxlen=max_length)

In [59]:
# Cross-validation
kf = KFold(n_splits=2, shuffle=True, random_state=42)
accuracy_scores = []
best_accuracy = 0
best_model = None

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import AdamW
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Xây dựng mô hình Bidirectional LSTM
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=128, input_length=1000))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    print(model)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import AdamW
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Xây dựng mô hình Bidirectional LSTM
    model = Sequential()
    model.add(Embedding(input_dim=20000, output_dim=128, input_length=1000))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # model.compile(loss='categorical_crossentropy', optimizer='adamw', metrics=['accuracy'])

    # # Huấn luyện mô hình
    # model.fit(X_train, y_train, epochs=70, batch_size=128, validation_data=(X_test, y_test), verbose=2)

    # # Đánh giá mô hình
    # loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    # accuracy_scores.append(accuracy)
    # print(f"Fold accuracy: {accuracy}")

    # # Lưu lại mô hình có độ chính xác cao nhất
    # if accuracy >= best_accuracy:
    #     best_accuracy = accuracy
    #     best_model = model

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
5/5 - 108s - 22s/step - accuracy: 0.0891 - loss: 2.4833 - val_accuracy: 0.1109 - val_loss: 2.4869
Epoch 2/70
5/5 - 120s - 24s/step - accuracy: 0.1345 - loss: 2.4753 - val_accuracy: 0.0992 - val_loss: 2.4886
Epoch 3/70
5/5 - 83s - 17s/step - accuracy: 0.1597 - loss: 2.4607 - val_accuracy: 0.0941 - val_loss: 2.4773
Epoch 4/70
5/5 - 71s - 14s/step - accuracy: 0.2252 - loss: 2.4194 - val_accuracy: 0.1933 - val_loss: 2.4259
Epoch 5/70
5/5 - 79s - 16s/step - accuracy: 0.2689 - loss: 2.3265 - val_accuracy: 0.2000 - val_loss: 2.3175
Epoch 6/70
5/5 - 91s - 18s/step - accuracy: 0.3160 - loss: 2.1551 - val_accuracy: 0.2790 - val_loss: 2.1497
Epoch 7/70
5/5 - 79s - 16s/step - accuracy: 0.3395 - loss: 1.9562 - val_accuracy: 0.4891 - val_loss: 1.9059
Epoch 8/70
5/5 - 71s - 14s/step - accuracy: 0.4756 - loss: 1.6974 - val_accuracy: 0.4773 - val_loss: 1.6856
Epoch 9/70
5/5 - 88s - 18s/step - accuracy: 0.5244 - loss: 1.4903 - val_accuracy: 0.5294 - val_loss: 1.6080
Epoch 10/70
5/5 - 82s - 16

In [21]:
import pickle
import os
MODEL_PATH = '/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT'
with open(os.path.join(MODEL_PATH, "LSTM_best_model.pkl"), 'wb') as model_file:
    pickle.dump(model, model_file)

In [61]:
import numpy as np
import os
import pickle
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, classification_report
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical
# import seaborn as sns
import matplotlib.pyplot as plt
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Load mô hình đã huấn luyện
MODEL_PATH = '/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT'
loaded_model = pickle.load(open(os.path.join(MODEL_PATH, "LSTM_best_model.pkl"), 'rb'))

# Dự đoán nhãn cho tập kiểm tra
y_pred_probs = loaded_model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# In ra confusion matrix
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

# In ra báo cáo phân loại
class_report = classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report LSTM:")
print(class_report)

# Trực quan hóa confusion matrix bằng Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [65]:
import numpy as np
import os
import pickle
import numpy as np
import tensorflow as tf
MODEL_PATH = '/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT'
loaded_model = pickle.load(open(os.path.join(MODEL_PATH,"LSTM_best_model.pkl"), 'rb'))
#Tạo hàm dự đoán
def predict_title(content):
  processed_title = preprocess_text(content)
  sequence = tokenizer.texts_to_sequences([processed_title])
  padded_sequence = pad_sequences(sequence, maxlen=max_length)
  prediction = loaded_model.predict(padded_sequence)
  predicted_class = np.argmax(prediction, axis=1)
  predicted_label = label_encoder. inverse_transform(predicted_class)
  return predicted_label[0]



In [17]:


noidung="Ba phần tư cổ phiếu trên thị trường giảm giá, lực bán dâng cao liên tục đẩy thanh khoản vượt 35.500 tỷ đồng trong phiên VN-Index sụt hơn 19 điểm.\
Dù tăng vượt 1.280 điểm vào hôm qua, VN-Index bị nhuộm đỏ ngay những phút đầu phiên giao dịch hôm nay, mất gần 8 điểm. Sau khoảng một tiếng, chỉ số này mới cải thiện lên tham chiếu, giằng co một lúc rồi lấy lại sắc xanh nhờ lực cầu tiếp ứng. Tuy nhiên áp lực chốt lời ở nhóm bluechip vẫn rất cao khiến chỉ số nhanh chóng rơi về dưới tham chiếu trước khi nghỉ trưa.\
Sang buổi chiều, chỉ số đại diện sàn HoSE khoác sắc đỏ liên tục. Nửa đầu buổi, chỉ số này chủ yếu giằng co quanh 1.270 điểm.\
Đến sau 14h, xu hướng bán tháo lan nhanh từ nhóm bluechip sang hàng loạt cổ phiếu. Chỉ số bất ngờ giảm một mạch về sát 1.250 điểm, tức thấp hơn tham chiếu khoảng 30 điểm. Tuy nhiên, thị trường nhanh chóng cải thiện trước khi bước vào phiên ATC.\
VN-Index đóng cửa ở gần 1.262 điểm, giảm hơn 19 điểm so với hôm trước. Đây là phiên điều chỉnh mạnh nhất hơn một tháng qua"
print(predict_title(noidung))


NameError: name 'predict_title' is not defined

In [ ]:
import gradio as gr

def predict_label(content):
  processed_title = preprocess_text(content)
  sequence = tokenizer.texts_to_sequences([processed_title])
  padded_sequence = pad_sequences(sequence, maxlen=500)
  prediction = loaded_model.predict(padded_sequence)
  predicted_class = np.argmax(prediction, axis=1)
  predicted_label = label_encoder. inverse_transform(predicted_class)
  return "Bài báo thuộc chủ đề: " + str(label_encoder.inverse_transform(predicted_label))

demo = gr.Interface(fn=predict_label, inputs="text", outputs="text")
demo.launch()